Import libraries

In [1]:
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
import numpy as np
import sys
import time
import math

## 2D rendering

Initializing settings

In [2]:
# Some api in the chain is translating the keystrokes to this binary string
# so instead of saying: ESCAPE = 27, we use the following.
ESCAPE = b'\x1b'

# Number of the glut window.
window = 0

def init():
    # Commands # 1
    glEnable(GL_POINT_SMOOTH) # 1.1 Smooths points
    glEnable(GL_LINE_SMOOTH)  # 1.2 Smooths the curves
    glEnable(GL_BLEND)        # 1.3 Combines the colors computed in the fragment shader with the colors in the colors buffers
    glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA) # 1.4 defines blending factors
    # Commands # 2
    glClearColor(1.0, 1.0, 1.0, 1.0) # 2.1 Sets widows/background color to white
    gluOrtho2D(-5.0, 5.0, -5.0, 5.0) # 2.2 Creates 2D orthonormal reference frame

Function to be plotted

In [3]:
def func_x2(input_vec):
    vertices = [[x, x*x] for x in input_vec]
    return np.array(vertices)

In [4]:
#2. Modify the code to draw a second simultaneous plot of a sine function.
def func_sin(input_vec): 
    vertices = [[x, np.sin(x)] for x in input_vec]
    return np.array(vertices)

In [5]:
#3. Modify the plot to draw a random white Gaussian process with zero mean and standard deviation σ = 0.1
def func_rand_normal(input_vec):
    vertices=[[x,np.random.normal(0, 0.1)] for x in input_vec]
    return np.array(vertices)

Plotting function in OpenGL

In [6]:
def plot_func():
    
    # Commands # 3
    glClear(GL_COLOR_BUFFER_BIT) # 3.1 clears z-buffer
    glColor3f(0.0, 0.0, 0.0)     # 3.2 defines black color
    glPointSize(3.0)             # 3.3 set point size
    glLineWidth(1.0)             # 3.4 set width for the curves
    
    # Commands # 4
    # Drawing reference frame
    glBegin(GL_LINES)
    glVertex2f(-5.0, 0.0)
    glVertex2f(5.0, 0.0)
    glVertex2f(0.0, 5.0)
    glVertex2f(0.0, -5.0)
    glEnd()
    
    # Set points to plot graphic
    v = np.linspace(-5.0,5.0,101)
    vertices1=func_x2(v)
    vertices2=func_sin(v)
    vertices3=func_rand_normal(v)
    
    # Adding square grid dx=dy=1
    axe = np.linspace(-5.0,5.0,15)
    for x in axe :
        glBegin(GL_LINES)
        glColor3f(0.0, 0.0, 0.0) 
        #x axis
        glVertex2f(x, -5)
        glVertex2f(x, 5)
        #y axis, here x and y have the same values (i.e. [-5,5])
        glVertex2f(-5, x)
        glVertex2f(5, x)
        glEnd()

    # Commands # 5
    # Connecting points to draw curves
    for i in range(len(vertices1)-1):
        glBegin(GL_LINES)
        glColor3f(0.8,0.2,0.2)
        
        # Square function
        glVertex2f(vertices1[i,0],vertices1[i,1])
        glVertex2f(vertices1[i+1,0],vertices1[i+1,1])
        
        # Sinus function
        glVertex2f(vertices2[i,0],vertices2[i,1])
        glVertex2f(vertices2[i+1,0],vertices2[i+1,1])
        
        # Random normal function 
        glVertex2f(vertices3[i,0],vertices3[i,1])
        glVertex2f(vertices3[i+1,0],vertices3[i+1,1])
        
        glEnd()
    
    # Commands # 6
    # Drawing points
    for i in range(len(vertices1)):
        glBegin(GL_POINTS)
        glColor3f(0.1,0.5,0.1)
        glVertex2fv(vertices1[i])
        glVertex2fv(vertices2[i])
        glVertex2fv(vertices3[i])
        glEnd()
    
    # Commands # 7
    # time.sleep(...)
    #Indicates that the computation is done with the current frame
    #and swaps the buffer to the next one 
    glutSwapBuffers()

Function that checks if a key has been pressed on the keyboard

In [7]:
# The function called whenever a key is pressed. Note the use of Python tuples to pass in: (key, x, y)  
def keyPressed(key, x, y):
    if key == ESCAPE:
        glutLeaveMainLoop()
        return

Main function with initialization, drawing and querying for external inputs (keyboard)

In [8]:
def main():
    global window
    glutInit(())
    glutInitDisplayMode(GLUT_SINGLE|GLUT_RGB)
    glutInitWindowPosition(50,50)
    glutInitWindowSize(500,500)
    glutCreateWindow(b"Function Plotter")
    glutDisplayFunc(plot_func)
    # When we are doing nothing, redraw the scene.
    glutIdleFunc(plot_func)
    # Register the function called when the keyboard is pressed.  
    glutKeyboardFunc(keyPressed)
    # Initialization
    init()
    # Main drawing loop
    glutMainLoop()

Executing main function

In [ ]:
# Print message to console, and kick off the main to get it rolling.
print("Hit ESC key to quit.")
main()

Hit ESC key to quit.


## ECG

In [2]:
# Load data
ecg= np.loadtxt('ecg.txt') 

In [3]:
len(ecg)
min(ecg) - 2047
max(ecg) - 2047

459.0

In [4]:

# Number of the glut window.
window = 0

def init():
    # Commands # 1
    glEnable(GL_POINT_SMOOTH) # 1.1 Smooths points
    glEnable(GL_LINE_SMOOTH)  # 1.2 Smooths the curves
    glEnable(GL_BLEND)        # 1.3 Combines the colors computed in the fragment shader with the colors in the colors buffers
    glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA) # 1.4 defines blending factors
    # Commands # 2
    glClearColor(1.0, 1.0, 1.0, 1.0) # 2.1 Sets widows/background color to white
    # amplitudes  range from −0.5 mV to 5 mV (x_axis)
    # i.e. -500V to 500V (x_axis)
    # amplitude from 0 to 4095 (y_axis)
    # 0mV--->2047
    # thus: min(ecg) --> min(ecg) - 2047 = - 60 
    #       max(ecg) --> max(ecg) - 2047 ~ 500
    gluOrtho2D(-500.0, 500.0,  -60, 459) # 2.2 Creates 2D orthonormal reference frame

In [5]:
global it
it=0

def plot_func():
    global it
    glClear(GL_COLOR_BUFFER_BIT)
    glColor3f(0.0, 0.0, 0.0)   
    glPointSize(3.0)             
    glLineWidth(1.0)            
    
    # square grid dx=dy=1
    #x-axis
    axe_x = np.linspace(-500,500,21)  #step = 0.01mV, nb of points 2047*0.01
    for x in axe_x :
        glBegin(GL_LINES)
        glColor3f(0.9, 0.71, 0.75) 
        #x axis
        glVertex2f(x, -60)
        glVertex2f(x, 459)
        glEnd()
    glLineWidth(0.5)
    
    #y-axis
    axe_y = np.linspace(-60,459,5) #we want a step == 0.01mv
    for y in axe_y :
        glBegin(GL_LINES)
        glColor3f(0.9, 0.71, 0.75) 
        #y axis
        glVertex2f(-500,y)
        glVertex2f(500,y)
        glEnd()
    glLineWidth(0.5)
    
    ################################################### 
    x = -500 #end point 
    while x<500:
        glBegin(GL_LINES)
        glColor3f(0.9, 0.71, 0.75) 
        glVertex2f(x,-60)
        glVertex2f(x,459)
        glEnd()
        x +=10 #step of 0.01mV=10V
    
    start_y=-60
    while(start_y<459):
        glBegin(GL_LINES)
        glColor3f(0.9, 0.71, 0.75)
        glVertex2f(-500, start_y)
        glVertex2f(500, start_y)
        glEnd()
        start_y+=10
    
    v_ecg=(ecg-2047) #rescaling
    vertices = []
    for i in range(0,1000): #0 to 1khz=1000hz
        vertices.append(v_ecg[i+it]) #adding the iterator allows to repeating and have those ECG waves

    for i in range(len(vertices)-1):
        glBegin(GL_LINES)
        glColor3f(0.0,0.0,0.0)
        glVertex2fv([i-500,vertices[i]])
        glVertex2fv([i-500,vertices[i+1]])        
        glEnd()
        
    it+=10
    glutSwapBuffers()

In [6]:
# The function called whenever a key is pressed. Note the use of Python tuples to pass in: (key, x, y)  
def keyPressed(key, x, y):
    if key == ESCAPE:
        glutLeaveMainLoop()
        return

In [7]:
def main():
    global window
    glutInit(())
    glutInitDisplayMode(GLUT_SINGLE|GLUT_RGB)
    glutInitWindowPosition(50,50)
    glutInitWindowSize(500,500)
    glutCreateWindow(b"ECG")
    glutDisplayFunc(plot_func)
    # When we are doing nothing, redraw the scene.
    glutIdleFunc(plot_func)
    # Register the function called when the keyboard is pressed.  
    glutKeyboardFunc(keyPressed)
    # Initialization
    init()
    # Main drawing loop
    glutMainLoop()

In [ ]:
# Print message to console, and kick off the main to get it rolling.
print("Hit ESC key to quit.")
main()

Hit ESC key to quit.
